# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
# nhl_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coursera/assets4/nhl.csv")
# cities=pd.read_html("/content/drive/MyDrive/Colab Notebooks/Coursera/assets4/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
for stuff in ['NFL','MLB','NBA','NHL']:
    cities[stuff]=cities[stuff].str.replace(r"\[.*\]","")
    cities[stuff]=cities[stuff].str.replace(r"[—]",'NaN')
    cities[stuff]=cities[stuff].str.replace("^\s*$",'NaN')
    cities[stuff]=cities[stuff].str.rstrip()
def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_df = nhl
    nhl_df=nhl_df[nhl_df['year']==2018]
    nhl_df=nhl_df[['team','W','L']]
    nhl_df['team']=nhl_df['team'].str.replace("\*","")
    nhl_df.drop([0,9,18,26],inplace=True)
    nhl_df['W']=nhl_df['W'].astype(int)
    nhl_df['L']=nhl_df['L'].astype(int)
    nhl_df['W/L']=nhl_df['W']/(nhl_df['W']+nhl_df['L'])
    nhl_area=cities[['Metropolitan area','NHL','Population (2016 est.)[8]']].replace('NaN',np.nan).dropna()
    nhl_area=nhl_area.rename(columns={'Population (2016 est.)[8]':'Population'})
    sp1=['Tampa Bay Lightning','New Jersey Devils','New York Islanders','New York Rangers','St. Louis Blues','San Jose Sharks','Los Angeles Kings']
    def GetCity1(team):
        if team in sp1:
            return team.split()[0]+' '+team.split()[1]
        else:
            return team.split()[0]
    nhl_df['Metropolitan area'] = nhl_df['team'].apply(lambda x: GetCity1(x))
    nhl_df=nhl_df[['Metropolitan area','W/L']]
    ny1=list(nhl_df.iloc[14:16]['W/L'])
    ny1.append(nhl_df.iloc[12]['W/L'])
    la1=list(nhl_df.iloc[24:25]['W/L'])
    la1.append(nhl_df.iloc[26]['W/L'])
    d2={'Metropolitan area':'New York City','W/L':np.mean(ny1)}
    d3={'Metropolitan area':'Los Angeles','W/L':np.mean(la1)}
    nhl_df=nhl_df.append([d2,d3], ignore_index = True)
    nhl_df.drop([12,14,15,24,26],inplace=True)
    nhl_area=nhl_area.set_index('Metropolitan area')
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Tampa Bay","Tampa Bay Area")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Florida","Miami–Fort Lauderdale")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Washington","Washington, D.C.")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Carolina","Raleigh")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Minnesota","Minneapolis–Saint Paul")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Colorado","Denver")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("San Jose","San Francisco Bay Area")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Arizona","Phoenix")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Dallas","Dallas–Fort Worth")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Vegas","Las Vegas")
    nhl_df=nhl_df.set_index('Metropolitan area')
    nhl_area=nhl_area[['Population']]
    nhl_area['Population']=nhl_area['Population'].astype(float)
    nhl_final=pd.merge(nhl_df,nhl_area,how='inner',left_index=True,right_index=True)

    population_by_region = nhl_final['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nhl_final['W/L'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
          
    [r,p] = stats.pearsonr(population_by_region, win_loss_by_region)
    return r
nhl_correlation()

0.012486162921209918

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
# nba_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coursera/assets4/nba.csv")
# cities=pd.read_html("/content/drive/MyDrive/Colab Notebooks/Coursera/assets4/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
for stuff in ['NFL','MLB','NBA','NHL']:
    cities[stuff]=cities[stuff].str.replace(r"\[.*\]","")
    cities[stuff]=cities[stuff].str.replace(r"[—]",'NaN')
    cities[stuff]=cities[stuff].str.replace("^\s*$",'NaN')
    cities[stuff]=cities[stuff].str.rstrip()

def nba_correlation():
    # YOUR CODE HERE
    nba_df = nba
    nba_df=nba_df[nba_df['year']==2018]
    nba_df=nba_df[['team','W','L']]
    nba_df['team']=nba_df['team'].str.replace("\*","")
    nba_df['team']=nba_df['team'].str.replace(r"\(.*\)","")
    nba_df['team']=nba_df['team'].str.rstrip()
    nba_df['W']=nba_df['W'].astype(int)
    nba_df['L']=nba_df['L'].astype(int)
    nba_df['W/L']=nba_df['W']/(nba_df['W']+nba_df['L'])
    nba_area=cities[['Metropolitan area','NBA','Population (2016 est.)[8]']].replace('NaN',np.nan).dropna()
    nba_area=nba_area.rename(columns={'Population (2016 est.)[8]':'Population'})
    sp2=['New York Knicks','Golden State Warriors','Oklahoma City Thunder','New Orleans Pelicans','San Antonio Spurs','Los Angeles Clippers','Los Angeles Lakers']
    def GetCity2(team):
        if team in sp2:
            return team.split()[0]+' '+team.split()[1]
        else:
            return team.split()[0]
    nba_df['Metropolitan area'] = nba_df['team'].apply(lambda x: GetCity2(x))
    nba_df=nba_df[['Metropolitan area','W/L']]
    ny2=list(nba_df.iloc[10:12]['W/L'])
    la2=list(nba_df.iloc[24:26]['W/L'])
    d2={'Metropolitan area':'New York City','W/L':np.mean(ny2)}
    d3={'Metropolitan area':'Los Angeles','W/L':np.mean(la2)}
    nba_df=nba_df.append([d2,d3], ignore_index = True)
    nba_df.drop([10,11,24,25],inplace=True)
    nba_area=nba_area.set_index('Metropolitan area')
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Indiana","Indianapolis")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Miami","Miami–Fort Lauderdale")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Washington","Washington, D.C.")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Golden State","San Francisco Bay Area")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Minnesota","Minneapolis–Saint Paul")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Utah","Salt Lake City")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Dallas","Dallas–Fort Worth")
    nba_df=nba_df.set_index('Metropolitan area')
    nba_area=nba_area[['Population']]
    nba_area['Population']=nba_area['Population'].astype(float)
    nba_final=pd.merge(nba_df,nba_area,how='inner',left_index=True,right_index=True)

    population_by_region = nba_final['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nba_final['W/L'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    [r,p] = stats.pearsonr(population_by_region, win_loss_by_region)
    return r
nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
# mlb_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coursera/assets4/mlb.csv")
# cities=pd.read_html("/content/drive/MyDrive/Colab Notebooks/Coursera/assets4/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
for stuff in ['NFL','MLB','NBA','NHL']:
    cities[stuff]=cities[stuff].str.replace(r"\[.*\]","")
    cities[stuff]=cities[stuff].str.replace(r"[—]",'NaN')
    cities[stuff]=cities[stuff].str.replace("^\s*$",'NaN')
    cities[stuff]=cities[stuff].str.rstrip()
    
def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df = mlb
    mlb_df=mlb_df[mlb_df['year']==2018]
    mlb_df=mlb_df[['team','W','L']]
    mlb_df['team']=mlb_df['team'].str.replace("\*","")
    mlb_df['team']=mlb_df['team'].str.replace(r"\(.*\)","")
    mlb_df['team']=mlb_df['team'].str.rstrip()
    mlb_df['W']=mlb_df['W'].astype(int)
    mlb_df['L']=mlb_df['L'].astype(int)
    mlb_df['W/L']=mlb_df['W']/(mlb_df['W']+mlb_df['L'])
    mlb_area=cities[['Metropolitan area','MLB','Population (2016 est.)[8]']].replace('NaN',np.nan).dropna()
    mlb_area=mlb_area.rename(columns={'Population (2016 est.)[8]':'Population'})
    sp3=['New York Yankees','Tampa Bay Rays','Kansas City Royals','Los Angeles Angels','St. Louis Cardinals','New York Mets','Los Angeles Dodgers','San Francisco Giants','San Diego Padres']
    def GetCity3(team):
        if team in sp3:
            return team.split()[0]+' '+team.split()[1]
        else:
             return team.split()[0]
    mlb_df['Metropolitan area'] = mlb_df['team'].apply(lambda x: GetCity3(x))
    mlb_df=mlb_df[['Metropolitan area','W/L']]
    ny3=list(mlb_df.iloc[1:2]['W/L'])
    ny3.append(mlb_df.iloc[18]['W/L'])
    la3=list(mlb_df.iloc[13:14]['W/L'])
    la3.append(mlb_df.iloc[25]['W/L'])
    sf3=list(mlb_df.iloc[11:12]['W/L'])
    sf3.append(mlb_df.iloc[28]['W/L'])
    ch3=list(mlb_df.iloc[8:9]['W/L'])
    ch3.append(mlb_df.iloc[21]['W/L'])
    d2={'Metropolitan area':'New York City','W/L':np.mean(ny3)}
    d3={'Metropolitan area':'Los Angeles','W/L':np.mean(la3)}
    d4={'Metropolitan area':'San Francisco Bay Area','W/L':np.mean(sf3)}
    d5={'Metropolitan area':'Chicago','W/L':np.mean(ch3)}
    mlb_df=mlb_df.append([d2,d3,d4,d5], ignore_index = True)
    mlb_df.drop([1,18,13,25,11,28,8,21],inplace=True)
    mlb_area=mlb_area.set_index('Metropolitan area')
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Tampa Bay","Tampa Bay Area")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Miami","Miami–Fort Lauderdale")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Washington","Washington, D.C.")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Minnesota","Minneapolis–Saint Paul")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Colorado","Denver")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Arizona","Phoenix")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Texas","Dallas–Fort Worth")
    mlb_df=mlb_df.set_index('Metropolitan area')
    mlb_area=mlb_area[['Population']]
    mlb_area['Population']=mlb_area['Population'].astype(float)
    mlb_final=pd.merge(mlb_df,mlb_area,how='inner',left_index=True,right_index=True)

    population_by_region = mlb_final['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = mlb_final['W/L'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]


    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    [r,p] = stats.pearsonr(population_by_region, win_loss_by_region)
    return r
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
# nfl_df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coursera/assets4/nfl.csv")
# cities=pd.read_html("/content/drive/MyDrive/Colab Notebooks/Coursera/assets4/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
for stuff in ['NFL','MLB','NBA','NHL']:
  cities[stuff]=cities[stuff].str.replace(r"\[.*\]","")
  cities[stuff]=cities[stuff].str.replace(r"[—]",'NaN')
  cities[stuff]=cities[stuff].str.replace("^\s*$",'NaN')
  cities[stuff]=cities[stuff].str.rstrip()
        
def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df = nfl
    nfl_df=nfl_df[nfl_df['year']==2018]
    nfl_df=nfl_df[['team','W','L']]
    nfl_df.drop([0,5,10,15,20,25,30,35],inplace=True)
    nfl_df['team']=nfl_df['team'].str.replace("[\*\+]","")
    nfl_df['team']=nfl_df['team'].str.rstrip()
    nfl_df['W']=nfl_df['W'].astype(int)
    nfl_df['L']=nfl_df['L'].astype(int)
    nfl_df['W/L']=nfl_df['W']/(nfl_df['W']+nfl_df['L'])
    nfl_area=cities[['Metropolitan area','NFL','Population (2016 est.)[8]']].replace('NaN',np.nan).dropna()
    nfl_area=nfl_area.rename(columns={'Population (2016 est.)[8]':'Population'})
    sp4=['New England Patriots','New York Jets','Kansas City Chiefs','Los Angeles Chargers','New York Giants','Green Bay Packers','New Orleans Saints','Tampa Bay Buccaneers','Los Angeles Rams','San Francisco 49ers']
    def GetCity4(team):
        if team in sp4:
            return team.split()[0]+' '+team.split()[1]
        else:
            return team.split()[0]
    nfl_df['Metropolitan area'] = nfl_df['team'].apply(lambda x: GetCity4(x))
    nfl_df=nfl_df[['Metropolitan area','W/L']]
    nfl_df=nfl_df.reset_index()
    del nfl_df['index']
    ny4=list(nfl_df.iloc[3:4]['W/L'])
    ny4.append(nfl_df.iloc[19]['W/L'])
    la4=list(nfl_df.iloc[13:14]['W/L'])
    la4.append(nfl_df.iloc[28]['W/L'])
    sf4=list(nfl_df.iloc[15:16]['W/L'])
    sf4.append(nfl_df.iloc[30]['W/L'])
    d2={'Metropolitan area':'New York City','W/L':np.mean(ny4)}
    d3={'Metropolitan area':'Los Angeles','W/L':np.mean(la4)}
    d4={'Metropolitan area':'San Francisco Bay Area','W/L':np.mean(sf4)}
    nfl_df=nfl_df.append([d2,d3,d4], ignore_index = True)
    nfl_df.drop([3,19,13,28,15,30],inplace=True)
    nfl_area=nfl_area.set_index('Metropolitan area')
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Tampa Bay","Tampa Bay Area")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Miami","Miami–Fort Lauderdale")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Washington","Washington, D.C.")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Minnesota","Minneapolis–Saint Paul")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Carolina","Charlotte")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Arizona","Phoenix")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Dallas","Dallas–Fort Worth")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("New England","Boston")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Tennessee","Nashville")
    nfl_df=nfl_df.set_index('Metropolitan area')
    nfl_area=nfl_area[['Population']]
    nfl_area['Population']=nfl_area['Population'].astype(float)
    nfl_final=pd.merge(nfl_df,nfl_area,how='inner',left_index=True,right_index=True)


    population_by_region = nfl_final['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = nfl_final['W/L'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    [r,p] = stats.pearsonr(population_by_region, win_loss_by_region)
    return r
nfl_correlation()

0.004922112149349471

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb=pd.read_csv("assets/mlb.csv")
nhl=pd.read_csv("assets/nhl.csv")
nba=pd.read_csv("assets/nba.csv")
nfl=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
for stuff in ['NFL','MLB','NBA','NHL']:
    cities[stuff]=cities[stuff].str.replace(r"\[.*\]","")
    cities[stuff]=cities[stuff].str.replace(r"[—]",'NaN')
    cities[stuff]=cities[stuff].str.replace("^\s*$",'NaN')
    cities[stuff]=cities[stuff].str.rstrip()
    
def sports_team_performance():
    # YOUR CODE HERE



    #NHL
    nhl_df=nhl
    nhl_df=nhl_df[nhl_df['year']==2018]
    nhl_df=nhl_df[['team','W','L']]
    nhl_df['team']=nhl_df['team'].str.replace("\*","")
    nhl_df.drop([0,9,18,26],inplace=True)
    nhl_df['W']=nhl_df['W'].astype(int)
    nhl_df['L']=nhl_df['L'].astype(int)
    nhl_df['W/L']=nhl_df['W']/(nhl_df['W']+nhl_df['L'])
    nhl_area=cities[['Metropolitan area','NHL','Population (2016 est.)[8]']].replace('NaN',np.nan).dropna()
    nhl_area=nhl_area.rename(columns={'Population (2016 est.)[8]':'Population'})
    sp1=['Tampa Bay Lightning','New Jersey Devils','New York Islanders','New York Rangers','St. Louis Blues','San Jose Sharks','Los Angeles Kings']
    def GetCity1(team):
        if team in sp1:
            return team.split()[0]+' '+team.split()[1]
        else:
            return team.split()[0]
    nhl_df['Metropolitan area'] = nhl_df['team'].apply(lambda x: GetCity1(x))
    nhl_df=nhl_df[['Metropolitan area','W/L']]
    ny1=list(nhl_df.iloc[14:16]['W/L'])
    ny1.append(nhl_df.iloc[12]['W/L'])
    la1=list(nhl_df.iloc[24:25]['W/L'])
    la1.append(nhl_df.iloc[26]['W/L'])
    d2={'Metropolitan area':'New York City','W/L':np.mean(ny1)}
    d3={'Metropolitan area':'Los Angeles','W/L':np.mean(la1)}
    nhl_df=nhl_df.append([d2,d3], ignore_index = True)
    nhl_df.drop([12,14,15,24,26],inplace=True)
    nhl_area=nhl_area.set_index('Metropolitan area')
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Tampa Bay","Tampa Bay Area")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Florida","Miami–Fort Lauderdale")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Washington","Washington, D.C.")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Carolina","Raleigh")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Minnesota","Minneapolis–Saint Paul")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Colorado","Denver")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("San Jose","San Francisco Bay Area")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Arizona","Phoenix")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Dallas","Dallas–Fort Worth")
    nhl_df['Metropolitan area']=nhl_df['Metropolitan area'].str.replace("Vegas","Las Vegas")
    nhl_df=nhl_df.set_index('Metropolitan area')
    nhl_area=nhl_area[['Population']]
    nhl_area['Population']=nhl_area['Population'].astype(float)
    nhl_final=pd.merge(nhl_df,nhl_area,how='inner',left_index=True,right_index=True)


    #NBA
    nba_df=nba
    nba_df=nba_df[nba_df['year']==2018]
    nba_df=nba_df[['team','W','L']]
    nba_df['team']=nba_df['team'].str.replace("\*","")
    nba_df['team']=nba_df['team'].str.replace(r"\(.*\)","")
    nba_df['team']=nba_df['team'].str.rstrip()
    nba_df['W']=nba_df['W'].astype(int)
    nba_df['L']=nba_df['L'].astype(int)
    nba_df['W/L']=nba_df['W']/(nba_df['W']+nba_df['L'])
    nba_area=cities[['Metropolitan area','NBA','Population (2016 est.)[8]']].replace('NaN',np.nan).dropna()
    nba_area=nba_area.rename(columns={'Population (2016 est.)[8]':'Population'})
    sp2=['New York Knicks','Golden State Warriors','Oklahoma City Thunder','New Orleans Pelicans','San Antonio Spurs','Los Angeles Clippers','Los Angeles Lakers']
    def GetCity2(team):
        if team in sp2:
            return team.split()[0]+' '+team.split()[1]
        else:
            return team.split()[0]
    nba_df['Metropolitan area'] = nba_df['team'].apply(lambda x: GetCity2(x))
    nba_df=nba_df[['Metropolitan area','W/L']]
    ny2=list(nba_df.iloc[10:12]['W/L'])
    la2=list(nba_df.iloc[24:26]['W/L'])
    d2={'Metropolitan area':'New York City','W/L':np.mean(ny2)}
    d3={'Metropolitan area':'Los Angeles','W/L':np.mean(la2)}
    nba_df=nba_df.append([d2,d3], ignore_index = True)
    nba_df.drop([10,11,24,25],inplace=True)
    nba_area=nba_area.set_index('Metropolitan area')
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Indiana","Indianapolis")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Miami","Miami–Fort Lauderdale")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Washington","Washington, D.C.")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Golden State","San Francisco Bay Area")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Minnesota","Minneapolis–Saint Paul")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Utah","Salt Lake City")
    nba_df['Metropolitan area']=nba_df['Metropolitan area'].str.replace("Dallas","Dallas–Fort Worth")
    nba_df=nba_df.set_index('Metropolitan area')
    nba_area=nba_area[['Population']]
    nba_area['Population']=nba_area['Population'].astype(float)
    nba_final=pd.merge(nba_df,nba_area,how='inner',left_index=True,right_index=True)

    #MLB
    mlb_df=mlb
    mlb_df=mlb_df[mlb_df['year']==2018]
    mlb_df=mlb_df[['team','W','L']]
    mlb_df['team']=mlb_df['team'].str.replace("\*","")
    mlb_df['team']=mlb_df['team'].str.replace(r"\(.*\)","")
    mlb_df['team']=mlb_df['team'].str.rstrip()
    mlb_df['W']=mlb_df['W'].astype(int)
    mlb_df['L']=mlb_df['L'].astype(int)
    mlb_df['W/L']=mlb_df['W']/(mlb_df['W']+mlb_df['L'])
    mlb_area=cities[['Metropolitan area','MLB','Population (2016 est.)[8]']].replace('NaN',np.nan).dropna()
    mlb_area=mlb_area.rename(columns={'Population (2016 est.)[8]':'Population'})
    sp3=['New York Yankees','Tampa Bay Rays','Kansas City Royals','Los Angeles Angels','St. Louis Cardinals','New York Mets','Los Angeles Dodgers','San Francisco Giants','San Diego Padres']
    def GetCity3(team):
        if team in sp3:
            return team.split()[0]+' '+team.split()[1]
        else:
             return team.split()[0]
    mlb_df['Metropolitan area'] = mlb_df['team'].apply(lambda x: GetCity3(x))
    mlb_df=mlb_df[['Metropolitan area','W/L']]
    ny3=list(mlb_df.iloc[1:2]['W/L'])
    ny3.append(mlb_df.iloc[18]['W/L'])
    la3=list(mlb_df.iloc[13:14]['W/L'])
    la3.append(mlb_df.iloc[25]['W/L'])
    sf3=list(mlb_df.iloc[11:12]['W/L'])
    sf3.append(mlb_df.iloc[28]['W/L'])
    ch3=list(mlb_df.iloc[8:9]['W/L'])
    ch3.append(mlb_df.iloc[21]['W/L'])
    d2={'Metropolitan area':'New York City','W/L':np.mean(ny3)}
    d3={'Metropolitan area':'Los Angeles','W/L':np.mean(la3)}
    d4={'Metropolitan area':'San Francisco Bay Area','W/L':np.mean(sf3)}
    d5={'Metropolitan area':'Chicago','W/L':np.mean(ch3)}
    mlb_df=mlb_df.append([d2,d3,d4,d5], ignore_index = True)
    mlb_df.drop([1,18,13,25,11,28,8,21],inplace=True)
    mlb_area=mlb_area.set_index('Metropolitan area')
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Tampa Bay","Tampa Bay Area")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Miami","Miami–Fort Lauderdale")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Washington","Washington, D.C.")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Minnesota","Minneapolis–Saint Paul")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Colorado","Denver")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Arizona","Phoenix")
    mlb_df['Metropolitan area']=mlb_df['Metropolitan area'].str.replace("Texas","Dallas–Fort Worth")
    mlb_df=mlb_df.set_index('Metropolitan area')
    mlb_area=mlb_area[['Population']]
    mlb_area['Population']=mlb_area['Population'].astype(float)
    mlb_final=pd.merge(mlb_df,mlb_area,how='inner',left_index=True,right_index=True)


    #NFL
    nfl_df=nfl
    nfl_df=nfl_df[nfl_df['year']==2018]
    nfl_df=nfl_df[['team','W','L']]
    nfl_df.drop([0,5,10,15,20,25,30,35],inplace=True)
    nfl_df['team']=nfl_df['team'].str.replace("[\*\+]","")
    nfl_df['team']=nfl_df['team'].str.rstrip()
    nfl_df['W']=nfl_df['W'].astype(int)
    nfl_df['L']=nfl_df['L'].astype(int)
    nfl_df['W/L']=nfl_df['W']/(nfl_df['W']+nfl_df['L'])
    nfl_area=cities[['Metropolitan area','NFL','Population (2016 est.)[8]']].replace('NaN',np.nan).dropna()
    nfl_area=nfl_area.rename(columns={'Population (2016 est.)[8]':'Population'})
    sp4=['New England Patriots','New York Jets','Kansas City Chiefs','Los Angeles Chargers','New York Giants','Green Bay Packers','New Orleans Saints','Tampa Bay Buccaneers','Los Angeles Rams','San Francisco 49ers']
    def GetCity4(team):
        if team in sp4:
            return team.split()[0]+' '+team.split()[1]
        else:
            return team.split()[0]
    nfl_df['Metropolitan area'] = nfl_df['team'].apply(lambda x: GetCity4(x))
    nfl_df=nfl_df[['Metropolitan area','W/L']]
    nfl_df=nfl_df.reset_index()
    del nfl_df['index']
    ny4=list(nfl_df.iloc[3:4]['W/L'])
    ny4.append(nfl_df.iloc[19]['W/L'])
    la4=list(nfl_df.iloc[13:14]['W/L'])
    la4.append(nfl_df.iloc[28]['W/L'])
    sf4=list(nfl_df.iloc[15:16]['W/L'])
    sf4.append(nfl_df.iloc[30]['W/L'])
    d2={'Metropolitan area':'New York City','W/L':np.mean(ny4)}
    d3={'Metropolitan area':'Los Angeles','W/L':np.mean(la4)}
    d4={'Metropolitan area':'San Francisco Bay Area','W/L':np.mean(sf4)}
    nfl_df=nfl_df.append([d2,d3,d4], ignore_index = True)
    nfl_df.drop([3,19,13,28,15,30],inplace=True)
    nfl_area=nfl_area.set_index('Metropolitan area')
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Tampa Bay","Tampa Bay Area")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Miami","Miami–Fort Lauderdale")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Washington","Washington, D.C.")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Minnesota","Minneapolis–Saint Paul")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Carolina","Charlotte")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Arizona","Phoenix")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Dallas","Dallas–Fort Worth")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("New England","Boston")
    nfl_df['Metropolitan area']=nfl_df['Metropolitan area'].str.replace("Tennessee","Nashville")
    nfl_df=nfl_df.set_index('Metropolitan area')
    nfl_area=nfl_area[['Population']]
    nfl_area['Population']=nfl_area['Population'].astype(float)
    nfl_final=pd.merge(nfl_df,nfl_area,how='inner',left_index=True,right_index=True)

    #FULL MERGE
    merge_1=pd.merge(nhl_final,nba_final,how='inner',left_index=True,right_index=True)
    merge_2=pd.merge(nhl_final,mlb_final,how='inner',left_index=True,right_index=True)
    merge_3=pd.merge(nhl_final,nfl_final,how='inner',left_index=True,right_index=True)
    merge_4=pd.merge(nba_final,mlb_final,how='inner',left_index=True,right_index=True)
    merge_5=pd.merge(nba_final,nfl_final,how='inner',left_index=True,right_index=True)
    merge_6=pd.merge(mlb_final,nfl_final,how='inner',left_index=True,right_index=True)
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    p_values['NHL']['NBA']=stats.ttest_rel(list(merge_1['W/L_x']), list(merge_1['W/L_y']))[1]
    p_values['NBA']['NHL']=p_values['NHL']['NBA']
    p_values['NHL']['MLB']=stats.ttest_rel(list(merge_2['W/L_x']), list(merge_2['W/L_y']))[1]
    p_values['MLB']['NHL']=p_values['NHL']['MLB']
    p_values['NHL']['NFL']=stats.ttest_rel(list(merge_3['W/L_x']), list(merge_3['W/L_y']))[1]
    p_values['NFL']['NHL']=p_values['NHL']['NFL']
    p_values['NBA']['MLB']=stats.ttest_rel(list(merge_4['W/L_x']), list(merge_4['W/L_y']))[1]
    p_values['MLB']['NBA']=p_values['NBA']['MLB']
    p_values['NBA']['NFL']=stats.ttest_rel(list(merge_5['W/L_x']), list(merge_5['W/L_y']))[1]
    p_values['NFL']['NBA']=p_values['NBA']['NFL']
    p_values['MLB']['NFL']=stats.ttest_rel(list(merge_6['W/L_x']), list(merge_6['W/L_y']))[1]
    p_values['NFL']['MLB']=p_values['MLB']['NFL']
    p_values
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
